In [68]:
import deepxde as dde
import numpy as np

In [69]:
a = dde.Variable(1.0)
b = dde.Variable(1.0)
c = dde.Variable(1.0)

In [70]:
geom = dde.geometry.TimeDomain(0, 40)

In [71]:
def Lorenz_system(t, w):
    x, y, z = w[:, 0:1], w[:, 1:2], w[:, 2:]
    dx_t = dde.grad.jacobian(w, t, i=0)
    dy_t = dde.grad.jacobian(w, t, i=1)
    dz_t = dde.grad.jacobian(w, t, i=2)
    return [
        dx_t - a * (y - x),
        dy_t - x * (b - z) - y, 
        dz_t - x * y - c * z,
    ]

In [72]:
ic1 = dde.icbc.IC(geom, lambda X: 1, lambda _, on_initial: on_initial, component=0)
ic2 = dde.icbc.IC(geom, lambda X: 1, lambda _, on_initial: on_initial, component=1)
ic3 = dde.icbc.IC(geom, lambda X: 1, lambda _, on_initial: on_initial, component=2)

In [73]:
def gen_traindata():
    data = np.load("/home/Alejandro/Gitlab/pinns/Lorenz_EQSyst/data.npz")
    return data["t"], data["w"]

In [75]:
observe_t, ob_w = gen_traindata()
observe_x = dde.icbc.PointSetBC(observe_t, ob_w[:, 0:1], component=0)
observe_y = dde.icbc.PointSetBC(observe_t, ob_w[:, 1:2], component=1)
observe_z = dde.icbc.PointSetBC(observe_t, ob_w[:, 2:3], component=2)

In [76]:
data = dde.data.PDE(
    geom,
    Lorenz_system,
    [ic1, ic2, ic3, observe_x, observe_y, observe_z],
    num_domain=400,
    num_boundary=2,
    anchors=observe_t,
)

/home/Alejandro/.local/lib/python3.9/site-packages/skopt/sampler/sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+1002=1002. 
  warnings.warn("The balance properties of Sobol' points require "


In [77]:
net = dde.nn.FNN([1] + [40] * 3 + [3], "tanh", "Glorot uniform")

In [79]:
model = dde.Model(data, net)
model.compile("adam", lr=0.001, external_trainable_variables=[a, b, c])
variable = dde.callbacks.VariableValue(
  [a, b, c], period=600, filename="variables.dat"
)

Compiling model...
Building feed-forward neural network...
'build' took 0.079934 s



/home/Alejandro/.local/lib/python3.9/site-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:103: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(
/home/Alejandro/.local/lib/python3.9/site-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
2022-05-12 18:16:39.282358: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-12 18:16:39.282403: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-12 18:16:39.282440: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lenovardo): /proc

'compile' took 0.594947 s



In [ ]:
losshistory, train_state = model.train(epochs=60000, callbacks=[variable])
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Initializing variables...
Training model...

Step      Train loss                                                                                    Test loss                                                                                     Test metric
0         [3.52e+00, 2.70e-01, 1.83e+00, 1.00e+00, 1.00e+00, 1.00e+00, 6.37e+01, 8.44e+01, 6.84e+02]    [3.52e+00, 2.70e-01, 1.83e+00, 1.00e+00, 1.00e+00, 1.00e+00, 6.37e+01, 8.44e+01, 6.84e+02]    []  
1000      [1.65e-01, 1.83e-01, 8.72e+00, 2.46e-04, 1.38e-02, 2.92e-01, 6.37e+01, 5.95e+01, 6.97e+01]    [1.65e-01, 1.83e-01, 8.72e+00, 2.46e-04, 1.38e-02, 2.92e-01, 6.37e+01, 5.95e+01, 6.97e+01]    []  
2000      [8.63e-02, 6.12e-01, 7.14e+00, 2.78e-02, 2.09e-02, 2.67e-01, 6.30e+01, 5.55e+01, 6.48e+01]    [8.63e-02, 6.12e-01, 7.14e+00, 2.78e-02, 2.09e-02, 2.67e-01, 6.30e+01, 5.55e+01, 6.48e+01]    []  
3000      [6.23e-01, 1.78e+00, 4.99e+00, 1.22e-01, 2.95e-01, 5.80e-02, 6.27e+01, 5.44e+01, 6.35e+01]    [6.23e-01, 1.78e+00, 4.99e+00, 1